# UltraInsight Project Code
## Description


### Setup

In [48]:
#setup
import numpy as np # linear algebra
import pandas as pd # data processing

#insert and read file
filename = "../archive (2)/TWO_CENTURIES_OF_UM_RACES.csv"
df = pd.read_csv(filename)
print(df)
df.head()

/var/folders/r6/y72_rzjs01v5mlmhssn_w27h0000gn/T/ipykernel_24890/1667280578.py:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


         Year of event Event dates                                Event name  \
0                 2018  06.01.2018                       Selva Costera (CHI)   
1                 2018  06.01.2018                       Selva Costera (CHI)   
2                 2018  06.01.2018                       Selva Costera (CHI)   
3                 2018  06.01.2018                       Selva Costera (CHI)   
4                 2018  06.01.2018                       Selva Costera (CHI)   
...                ...         ...                                       ...   
7461190           1995  00.00.1995                 La SainteLyon 65 km (FRA)   
7461191           1995  00.00.1995                 La SainteLyon 65 km (FRA)   
7461192           1995  00.00.1995  Szombathely 24 hours running Race  (HUN)   
7461193           1995  00.00.1995  Szombathely 24 hours running Race  (HUN)   
7461194           1995  00.00.1995  Szombathely 24 hours running Race  (HUN)   

        Event distance/length  Event nu

,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
0,2018,06.01.2018,Selva Costera (CHI),50km,22,4:51:39 h,Tnfrc,CHI,1978.0,M,M35,10.286,0
1,2018,06.01.2018,Selva Costera (CHI),50km,22,5:15:45 h,Roberto Echeverría,CHI,1981.0,M,M35,9.501,1
2,2018,06.01.2018,Selva Costera (CHI),50km,22,5:16:44 h,Puro Trail Osorno,CHI,1987.0,M,M23,9.472,2
3,2018,06.01.2018,Selva Costera (CHI),50km,22,5:34:13 h,Columbia,ARG,1976.0,M,M40,8.976,3
4,2018,06.01.2018,Selva Costera (CHI),50km,22,5:54:14 h,Baguales Trail,CHI,1992.0,M,M23,8.469,4


### Data Preprocessing and Cleaning

In [49]:
# select most recent 10 years
year_cutoff = df['Year of event'].max() - 10
print(len(df))

df = df.dropna()
df = df.drop_duplicates()
print(len(df))

year_inds = df['Year of event'] >= year_cutoff
df = df[year_inds]
print(len(df))

print(df["Event distance/length"])

7461195
4419663
2934506
0          50km
1          50km
2          50km
3          50km
4          50km
           ... 
6823922    50km
6823923    50km
6823924    50km
6823925    50km
6823926    50km
Name: Event distance/length, Length: 2934506, dtype: object


In [50]:
#select km and mi races only, not timed ones
distance = df["Event distance/length"]
non_time_km = distance.str.contains('km', regex=False)
non_time_mi = distance.str.endswith('mi')

#setup 2 different dataframes, one with km and one with mi races
df1 = df[non_time_km]
df2 = df[non_time_mi]
print(df1)
print(df2)
print(len(df1))
print(len(df2))

#Take the mi, mile, and miles races and strip them of their ending letters
conversion_kit = df2["Event distance/length"]
print(conversion_kit)
miles = conversion_kit.str.rstrip('miles')
miles = miles.str.rstrip('mile')
miles = miles.str.rstrip('mi')
print(miles)

#manual cleaning up of commas to convert to string floats
miles = miles.str.replace('37,5','37.5')
miles = miles.str.replace('52,5','52.5')
miles = miles.str.replace('52,4','52.4')
miles = miles.str.replace('67,5','67.5')

#convert mi to km
mi_to_km = 1.60934 * miles.astype(float)
print('In kms')
print(mi_to_km)


         Year of event Event dates                         Event name  \
0                 2018  06.01.2018                Selva Costera (CHI)   
1                 2018  06.01.2018                Selva Costera (CHI)   
2                 2018  06.01.2018                Selva Costera (CHI)   
3                 2018  06.01.2018                Selva Costera (CHI)   
4                 2018  06.01.2018                Selva Costera (CHI)   
...                ...         ...                                ...   
6823922           2015  09.10.2015  West Virginia Trilogy 50 km (USA)   
6823923           2015  09.10.2015  West Virginia Trilogy 50 km (USA)   
6823924           2015  09.10.2015  West Virginia Trilogy 50 km (USA)   
6823925           2015  09.10.2015  West Virginia Trilogy 50 km (USA)   
6823926           2015  09.10.2015  West Virginia Trilogy 50 km (USA)   

        Event distance/length  Event number of finishers Athlete performance  \
0                        50km              

In [51]:
#remove km ending from km df
conversion = df1["Event distance/length"]
#conversion = conversion.rename(columns = {"Event distance/length" : "Event distance"})
#print(conversion)
km = conversion

# non_time_km = distance.str.contains('km', regex=False)
good_inds = ~km.str.contains('.Etappen', regex=True) & \
    ~km.str.contains('.Etap', regex=True) & \
    ~km.str.contains(' Sp', regex=False) & \
    ~km.str.contains('.x.', regex=True) & \
    ~km.str.contains('.stag', regex=True)
print(len(good_inds))
print(len(df1))
# km = km[~km.str.contains('.Etappen', regex=True)]
# km = km[~km.str.contains('.Etap', regex=True)]
# km = km[~km.str.contains(' Sp', regex=False)]
# km = km[~km.str.contains('.x.', regex=True)]
# km = km[~km.str.contains('.stag', regex=True)]
km = km[good_inds]

km = km.str.rstrip('kilometers')
km = km.str.rstrip('kilometer')
km = km.str.rstrip('km')
km = km.str.rstrip('km ')

# km to float
km = km.astype(float)
print(km.mean())
print(mi_to_km.mean())
df1 = df1[good_inds]

# replace column with numeric values
# print(len(df1))
df1['Event distance/length'] = km
df2['Event distance/length'] = mi_to_km
print(df1['Event distance/length'].mean())
print(df2['Event distance/length'].mean())

giga_chad_df = pd.concat([df1, df2])
print(giga_chad_df['Event distance/length'].mean())
giga_chad_df = giga_chad_df.rename(columns = {"Event distance/length" : "Event distance"})
df = giga_chad_df
#combine both dfs
# merged_km = [mi_to_km, km]
# # distance = pd.concat(merged_km)
# #distance = distance.rename(columns = {"Event distance/length" : "Event distance"})
# print(distance.mean())

# #reintegrate back into main dataframe
# df["Event distance/length"].update(distance)
#df["Event distance/length"] = df["Event distance/length"].method(distance)
# print(df)
# acceptable_dist_inds = distance.index
# print(acceptable_dist_inds)
# print(distance)
# df = df[acceptable_dist_inds]
# print(df["Event distance/length"])
# print(df.index.max())
# print(acceptable_dist_inds.max())
# df[df.index.isin(acceptable_dist_inds)]


2377292
2377292
70.45161105751632
103.45027418057403
70.45161105751632
103.45027418057403


/var/folders/r6/y72_rzjs01v5mlmhssn_w27h0000gn/T/ipykernel_24890/1814451567.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Event distance/length'] = mi_to_km


74.60960228851859


In [52]:
#rename age category
df = df.rename(columns = {"Athlete age category" : "Athlete age"})

#remove letters from in front of ages (W,MU,M)
rm_age_cat = df["Athlete age"]
print(rm_age_cat)
age = rm_age_cat.str.lstrip('MU')
age = age.str.lstrip('M')
age = age.str.lstrip('W')
age = age.str.lstrip('F')
age = age.str.lstrip('U')
print(age)
age = age.astype(int)

#reintegrate back into main dataframe
df["Athlete age"].update(age)
print(df)
print(df["Athlete age"].mean())


0          M35
1          M35
2          M23
3          M40
4          M23
          ... 
6822564    W40
6822565    M45
6822566    W45
6822567    M23
6822568    M40
Name: Athlete age, Length: 2691103, dtype: object
0          35
1          35
2          23
3          40
4          23
           ..
6822564    40
6822565    45
6822566    45
6822567    23
6822568    40
Name: Athlete age, Length: 2691103, dtype: object
         Year of event     Event dates                         Event name  \
0                 2018      06.01.2018                Selva Costera (CHI)   
1                 2018      06.01.2018                Selva Costera (CHI)   
2                 2018      06.01.2018                Selva Costera (CHI)   
3                 2018      06.01.2018                Selva Costera (CHI)   
4                 2018      06.01.2018                Selva Costera (CHI)   
...                ...             ...                                ...   
6822564           2015  09.-11.10.2015  Bi

### Data Analysis

In [53]:


# Convert "Athlete performance" to float
df["Athlete performance"] = df["Athlete performance"].str.replace(':', '.')
df["Athlete performance"] = df["Athlete performance"].str.replace(' ', '0')
df["Athlete performance"] = df["Athlete performance"].str.rstrip('hour')
df["Athlete performance"] = df["Athlete performance"].str.rstrip('hr')
df["Athlete performance"] = df["Athlete performance"].str.rstrip('h')

# Remove entries containing 'd'
df = df[~df["Athlete performance"].str.contains('d', regex=True)]

# Split "Athlete performance" into hours, minutes, and seconds
df["Athlete performance"] = df["Athlete performance"].str.replace('.', '|', regex=False)
df[["Hours", "Minutes", "Seconds"]] = df["Athlete performance"].str.split(pat='|', expand=True)

# Convert time components to float and calculate total time in minutes
df[["Hours", "Minutes", "Seconds"]] = df[["Hours", "Minutes", "Seconds"]].astype(float)
time_in_mins = 60.0 * df["Hours"] + df["Minutes"] + df["Seconds"] / 60.0

# Calculate average pace per race distance
avg_pace = time_in_mins / df["Event distance"] 

df["Pace(min/km)"] = avg_pace


In [67]:
# Calculate average pace for specific distances
avg_pace_50km = time_in_mins / df[df["Event distance"].between(50, 51)]["Event distance"] 
avg_pace_80km = time_in_mins / df[df["Event distance"].between(80, 81)]["Event distance"] 
avg_pace_100km = time_in_mins / df[df["Event distance"].between(100, 101)]["Event distance"] 
avg_pace_161km = time_in_mins / df[df["Event distance"].between(160, 161)]["Event distance"] 


# Print average paces
print("Average pace for 50 km race:", avg_pace_50km.mean())
print("Average pace for 80 km race:", avg_pace_80km.mean())
print("Average pace for 100 km race:", avg_pace_100km.mean())
print("Average pace for 161 km race:", avg_pace_161km.mean())



Average pace for 50 km race: 8.127476346179167
Average pace for 80 km race: 8.874833084624413
Average pace for 100 km race: 9.12799018544078
Average pace for 161 km race: 10.45223110247668


In [66]:
#top 20% paces per race distance

# Define the race distance ranges
race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# Create an empty DataFrame to store the top 20% of paces for each range
top_20_percent_df = pd.DataFrame()

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Calculate the number of rows corresponding to the top 20% of paces
    top_20_percent_rows = int(0.2 * len(filtered_df))
    
    # Sort the filtered DataFrame by pace in descending order
    sorted_df = filtered_df.sort_values(by="Pace(min/km)", ascending=False)
    
    # Take the top 20% of rows from the sorted DataFrame
    top_20_df = sorted_df.head(top_20_percent_rows)
    
    # Append the top 20% of paces to the DataFrame
    top_20_percent_df = pd.concat([top_20_percent_df, top_20_df])

# Print the DataFrame containing the top 20% of paces for each range
print(top_20_percent_df)

# Calculate the average pace for each race distance group
average_pace_per_distance_list = []

for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df = top_20_percent_df[(top_20_percent_df["Event distance"] >= range_start) & 
                                     (top_20_percent_df["Event distance"] < range_end)]
    
    # Calculate the average pace for the top 20% of paces in the current distance range
    average_pace = filtered_df["Pace(min/km)"].mean()
    
    # Store the results in a tuple
    result = ((range_start, range_end), average_pace)
    
    # Append the result to the list
    average_pace_per_distance_list.append(result)

# Print the average pace per distance range
for result in average_pace_per_distance_list:
    print("Distance Range:", result[0], "Average Pace:", result[1])




         Year of event     Event dates  \
926118            2016      21.05.2016   
5704200           2013  16.-19.05.2013   
3113795           2021      13.11.2021   
2617721           2020      22.02.2020   
2617720           2020      22.02.2020   
...                ...             ...   
6681273           2015  19.-20.09.2015   
2945097           2021  17.-18.09.2021   
1075522           2016  23.-24.09.2016   
52385             2018  17.-19.02.2018   
6810948           2015  16.-17.10.2015   

                                                Event name  Event distance  \
926118        Knock on Wood Ultra - Forest Freak 50k (USA)          50.000   
5704200                 3 Days at the Fair 50km Race (USA)          50.000   
3113795                  Grand Ridge 50 Km Trail Run (USA)          50.000   
2617721                          Lord Hill Trail Run (USA)          50.000   
2617720                          Lord Hill Trail Run (USA)          50.000   
...                        

In [77]:
#stdev for outliers of pace per race distance

# Define the race distance ranges
race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# Create empty lists to store lower and upper bound ranges
lower_bound_ranges = []
upper_bound_ranges = []

# Iterate over each race distance range
for range_start, range_end in race_distance_ranges:
    # Filter the DataFrame for the specified race distance range
    filtered_df1 = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
    # Calculate the mean and standard deviation of pace for the filtered DataFrame
    mean_pace = filtered_df1["Pace(min/km)"].mean()
    std_dev_pace = filtered_df1["Pace(min/km)"].std()
    
    # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
    lower_bound = mean_pace - 1.5 * std_dev_pace
    upper_bound = mean_pace + 1.5 * std_dev_pace
    
    # Add the race distance and lower bound range to the lower_bound_ranges list
    lower_bound_ranges.append((range_start, range_end, lower_bound))
    
    # Add the race distance and upper bound range to the upper_bound_ranges list
    upper_bound_ranges.append((range_start, range_end, upper_bound))

# Convert the lists into DataFrames
lower_bounds_df = pd.DataFrame(lower_bound_ranges, columns=["Start", "End", "Lower Bound"])
upper_bounds_df = pd.DataFrame(upper_bound_ranges, columns=["Start", "End", "Upper Bound"])

# Print the DataFrames
print("Lower Bound Ranges:")
print(lower_bounds_df)
print("\nUpper Bound Ranges:")
print(upper_bounds_df)










# # Define the race distance ranges
# race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# # Create empty lists to store outlier information
# lower_bounds = []
# upper_bounds = []

# # Iterate over each race distance range
# for range_start, range_end in race_distance_ranges:
#     # Filter the DataFrame for the specified race distance range
#     filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
#     # Calculate the mean and standard deviation of pace for the filtered DataFrame
#     mean_pace = filtered_df["Pace(min/km)"].mean()
#     std_dev_pace = filtered_df["Pace(min/km)"].std()
    
#     # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
#     lower_bound = mean_pace - 1.5 * std_dev_pace
#     upper_bound = mean_pace + 1.5 * std_dev_pace
    
#     # Add the race distance and lower bound to the lower_bounds list
#     lower_bounds.append({"Event distance": f"{range_start}-{range_end}", "Lower Bound": lower_bound})
    
#     # Add the race distance and upper bound to the upper_bounds list
#     upper_bounds.append({"Event distance": f"{range_start}-{range_end}", "Upper Bound": upper_bound})

# # Convert the lists into DataFrames
# lower_bounds_df = pd.DataFrame(lower_bounds)
# upper_bounds_df = pd.DataFrame(upper_bounds)

# # Print the DataFrames
# print("Lower Bounds:")
# print(lower_bounds_df)
# print("\nUpper Bounds:")
# print(upper_bounds_df)










# # Define the race distance ranges
# race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# # Create an empty list to store outlier information
# outliers_info = []

# # Iterate over each race distance range
# for range_start, range_end in race_distance_ranges:
#     # Filter the DataFrame for the specified race distance range
#     filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
#     # Calculate the mean and standard deviation of pace for the filtered DataFrame
#     mean_pace = filtered_df["Pace(min/km)"].mean()
#     std_dev_pace = filtered_df["Pace(min/km)"].std()
    
#     # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
#     lower_bound = mean_pace - 1.5 * std_dev_pace
#     upper_bound = mean_pace + 1.5 * std_dev_pace
    
#     # Create a range of values for outliers
#     outlier_range = f"{lower_bound:.2f}-{upper_bound:.2f}"
    
#     # Add the race distances and outlier ranges to the outliers_info list
#     outliers_info.append({"Event distance": f"{range_start}-{range_end}", "Outlier Range": outlier_range})

# # Convert the outliers_info list into a DataFrame
# outliers_pace_range_df = pd.DataFrame(outliers_info)

# # Print the outliers_pace_range_df DataFrame
# print(outliers_pace_range_df)





# # Define the race distance ranges
# race_distance_ranges = [(50, 51), (80, 81), (100, 101), (160, 161)]

# # Create an empty DataFrame to store outliers
# outliers_df = pd.DataFrame(columns=df.columns)

# # Iterate over each race distance range
# for range_start, range_end in race_distance_ranges:
#     # Filter the DataFrame for the specified race distance range
#     filtered_df = df[(df["Event distance"] >= range_start) & (df["Event distance"] < range_end)]
    
#     # Calculate the mean and standard deviation of pace for the filtered DataFrame
#     mean_pace = filtered_df["Pace(min/km)"].mean()
#     std_dev_pace = filtered_df["Pace(min/km)"].std()
    
#     # Define the lower and upper bounds for identifying outliers (e.g., 1.5 times the standard deviation from the mean)
#     lower_bound = mean_pace - 1.5 * std_dev_pace
#     upper_bound = mean_pace + 1.5 * std_dev_pace
    
#     # Identify outliers within the specified range and append them to the outliers DataFrame
#     outliers_within_range = filtered_df[(filtered_df["Pace(min/km)"] < lower_bound) | (filtered_df["Pace(min/km)"] > upper_bound)]
#     outliers_df = pd.concat([outliers_df, outliers_within_range])

# # Print the outliers DataFrame
# print(outliers_df)


Lower Bound Ranges:
   Start  End  Lower Bound
0     50   51     4.864022
1     80   81     5.815469
2    100  101     5.047724
3    160  161     7.289924

Upper Bound Ranges:
   Start  End  Upper Bound
0     50   51    11.293370
1     80   81    11.860050
2    100  101    12.901692
3    160  161    13.591164
